# Procesamiento de pdfs

# columna id, partido politico,content

## 1 pag useless

In [10]:
import os
import csv
from PyPDF2 import PdfReader
import sys

# Incrementar el límite del tamaño del campo
csv.field_size_limit(10**6)

# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "MOVIMIENTO DE UNIDAD PLURINACIONAL PACHAKUTIK _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras 3 páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=3):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")



Datos agregados al archivo CSV: candidatos.csv


## 2 pag useless

In [11]:
import os
import csv
from PyPDF2 import PdfReader

csv.field_size_limit(10**6)
# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "PARTIDO SOCIEDAD PATRIÓTICA  21 DE ENERO _Plan de trabajo_.pdf",
    "PARTIDO UNIDAD POPULAR _Plan de trabajo_.pdf",
    "PARTIDO SOCIALISTA ECUATORIANO _Plan de trabajo_.pdf",
    "MOVIMIENTO DEMOCRACIA SÍ _Plan de trabajo_.pdf",
    "PARTIDO AVANZA _Plan de trabajo_.pdf",
    "PARTIDO SOCIAL CRISTIANO _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=2):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


## 3 pag useless

In [ ]:
import os
import csv
from PyPDF2 import PdfReader

csv.field_size_limit(10**6)
# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA _PID_ _Plan de trabajo_.pdf",
    "MOVIMIENTO ACCION DEMOCRATICA NACIONAL, ADN _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=3):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


## 4

In [14]:
import os
import csv
from PyPDF2 import PdfReader

csv.field_size_limit(10**6)
# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "MOVIMIENTO CENTRO DEMOCRÁTICO _Plan de trabajo_.pdf",
    "MOVIMIENTO CONSTRUYE _Plan de trabajo_.pdf",
    "MOVIMIENTO CREO, CREANDO OPORTUNIDADES _Plan de trabajo_.pdf",
    "MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPENDIENTE GENERANDO OPORTUNIDADES _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=4):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


# 5

In [15]:
import os
import csv
from PyPDF2 import PdfReader

csv.field_size_limit(10**6)
# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "PARTIDO IZQUIERDA DEMOCRÁTICA _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=5):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


# 7

In [16]:
import os
import csv
from PyPDF2 import PdfReader

csv.field_size_limit(10**6)
# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "PARTIDO SOCIEDAD UNIDA MÁS ACCIÓN, SUMA _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=7):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


# 8

In [17]:
import os
import csv
from PyPDF2 import PdfReader

csv.field_size_limit(10**6)
# Directorio donde están los archivos PDF
pdf_directory = "./data/"
output_csv = "candidatos.csv"

# Lista de archivos específicos a procesar
specific_files = [
    "REVOLUCIÓN CIUDADANA - RETO _Plan de trabajo_.pdf"
]

# Crear una lista para almacenar los datos
data = []

# Contador para el ID inicial basado en el último ID en el archivo CSV
def get_last_id(csv_path):
    if not os.path.exists(csv_path):
        return 1
    with open(csv_path, mode="r", encoding="utf-8") as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Saltar la cabecera
        rows = list(reader)
        if not rows:
            return 1
        return int(rows[-1][0]) + 1

# Función para extraer texto del PDF excluyendo las primeras páginas
def extract_text_excluding_pages(pdf_path, exclude_pages=8):
    reader = PdfReader(pdf_path)
    extracted_text = ""
    for i in range(exclude_pages, len(reader.pages)):
        extracted_text += reader.pages[i].extract_text() + "\n"
    return extracted_text.strip()

# Obtener el ID inicial
file_id = get_last_id(output_csv)

# Recorrer la lista de archivos específicos
for file_name in specific_files:
    # Construir la ruta completa del archivo
    pdf_path = os.path.join(pdf_directory, file_name)
    
    # Verificar si el archivo existe
    if os.path.exists(pdf_path):
        # Procesar el nombre del archivo
        processed_name = file_name.replace("_Plan de trabajo_", "").replace(".pdf", "")
        
        # Extraer el contenido del PDF
        content = extract_text_excluding_pages(pdf_path)
        
        # Agregar los datos a la lista
        data.append([file_id, processed_name, content])
        file_id += 1
    else:
        print(f"Archivo no encontrado: {file_name}")

# Agregar los datos al archivo CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # No escribimos la cabecera, solo los nuevos datos
    writer.writerows(data)

print(f"Datos agregados al archivo CSV: {output_csv}")


Datos agregados al archivo CSV: candidatos.csv


# Limpiar la columna content

In [26]:
import csv
import re

# Función para limpiar el contenido
def clean_content(text):
    # Eliminar caracteres específicos no deseados
    text = re.sub(r'[x>ĂĨŽƌŵĂĐŝſŶƐĞĐĞŶƚƌĂƌĄĞŶĚĞƐĂƌƌŽůůĂƌŝŶĚŝǀŝĚƵŽƐĐŽŶƐĐŝĞŶƚĞƐ͕ƋƵĞďƵƐƋƵĞŶĚƵƌĂŶƚĞƚŽĚĂƐƵǀŝĚĂĂƉŽƌƚĂƌĂůĂƐŽĐŝĞĚĂĚĐŽŶƚƌĂďĂũŽĐƌĞĂƚŝǀŽ͕ŝŶŶŽǀĂĐŝŽŶĞƐ͕ĐŽŶƐĞŶƐŝďŝůŝĚĂĚĂŵďŝĞŶƚĂůƋƵĞůĞůůĞǀĞĂĞŵƉƌĞŶĚĞƌƐƵƐĂĐĐŝŽŶĞƐƉƌŽĚƵĐƚŝǀĂƐĚĞŵĂŶĞƌĂĂŵŝŐĂďůĞĐŽŶĞůĞŶƚŽƌŶŽǇĐŽƌƌĞƐƉŽŶĚŝĞŶƚĞƐĂůĚ]', '', text)

    # Eliminar viñetas comunes como puntos y guiones
    text = re.sub(r"[\u2022\u25CB\u2023\u2219\u2022\u25AA\u25B6\u25B7\u25C6\u2043\u25B8\u25BB\u2660\u25FE\u25FB]", "", text)
    
    # Eliminar las enumeraciones (números seguidos de punto)
    text = re.sub(r'^\d+\.', '', text)  # Elimina enumeraciones al inicio de cada línea
    text = re.sub(r'\n\d+\.', '\n', text)  # Elimina enumeraciones que están en medio de las líneas
    
    # Reemplazar múltiples espacios con uno solo
    text = re.sub(r'\s+', ' ', text)

    # Eliminar las enumeraciones (números seguidos de punto) en cualquier parte del texto
    
    text = re.sub(r'\b\d+\.\s*', '', text)  # Elimina números seguidos de punto
    # Eliminar espacios antes y después del texto
    text = text.strip()
    
    # Eliminar caracteres no imprimibles (como saltos de línea innecesarios)
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Eliminar caracteres no ASCII
    
    # Eliminar números sueltos
    text = re.sub(r'\b\d+\b', '', text)  # Eliminar números aislados
    
    # Eliminar caracteres no alfanuméricos (eliminar puntuación y otros caracteres)
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar todo lo que no sea letras y números

    # Eliminar las comas
    text = text.replace(',', '')

    # Eliminar saltos de línea y reemplazarlos por un espacio, manteniendo el texto en un solo párrafo
    text = text.replace('\n', ' ').replace('\r', ' ')

    return text

# Leer el archivo CSV, limpiar la columna "content" y guardarlo de nuevo
input_csv = "candidatos.csv"
output_csv = "propuestasLimpias.csv"

# Abrir el archivo CSV original
with open(input_csv, mode="r", encoding="utf-8") as csv_file:
    reader = csv.DictReader(csv_file)
    rows = list(reader)

# Limpiar la columna "content"
for row in rows:
    if 'content' in row:  # Verificar si la columna 'content' existe
        row['content'] = clean_content(row['content'])

# Escribir el archivo CSV limpio
with open(output_csv, mode="w", newline="", encoding="utf-8") as csv_file:
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()  # Escribir la cabecera
    writer.writerows(rows)

print(f"Archivo CSV limpio guardado como: {output_csv}")


Archivo CSV limpio guardado como: propuestasLimpias.csv
